In [1]:
import os,sys
sys.path.append("../../")
from analysis_common import *

In [2]:
from root_common import *

Welcome to JupyROOT 6.24/06


In [3]:
#make a canvas we can use by default
c = ROOT.TCanvas()

In [4]:
#some useful functions
from analysis_helpers import *

In [5]:
#makes plotting more interactive in notebook
%jsroot on

In [6]:
#some useful early definitions
lep_accept_pt = 0.4 #GeV
lep_accept_angle = radians(40)

hadron_accept_angle = radians(40)
hadron_accept_ke = 0.060 #GeV

In [16]:
#NOTE YOU WILL NEED TO MODIFY THIS PARENT DIRECTORY!
dir_files_GEM21_11b = "/Data/ldmx_eTi_4GeV_GEM21_11b_00"
dir_files_G18_02a = "/Data/ldmx_eTi_4GeV_G18_02a_00"

In [17]:
#create a list of the files that we have
gst_files_GEM21_11b = glob.glob(f"{dir_files_GEM21_11b}/*gst.root")
gst_files_G18_02a = glob.glob(f"{dir_files_G18_02a}/*gst.root")

In [18]:
for f in gst_files_GEM21_11b[:10]: print(f)

/Data/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run1.gst.root
/Data/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run10.gst.root
/Data/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run2.gst.root
/Data/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run3.gst.root
/Data/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run4.gst.root
/Data/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run5.gst.root
/Data/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run6.gst.root
/Data/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run7.gst.root
/Data/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run8.gst.root
/Data/ldmx_eTi_4GeV_GEM21_11b_00/ldmx_eTi_4GeV_GEM21_11b_00_run9.gst.root


In [19]:
#defined in analysis_helpers.py ... creates a TChain from our list of files
gst_chain_GEM21_11b = create_gst_chain(gst_files_GEM21_11b,verbose=True)

Created gst chain from 10 files with 1000000 total events.


In [20]:
gst_chain_G18_02a = create_gst_chain(gst_files_G18_02a,verbose=True)

Created gst chain from 10 files with 1000000 total events.


In [21]:
#creates a dataframe from our TChain
# see https://root.cern/doc/master/classROOT_1_1RDataFrame.html for examples
df_gst_GEM21_11b_all = ROOT.RDataFrame(gst_chain_GEM21_11b)

In [22]:
n_entries = df_gst_GEM21_11b_all.Count().GetValue()
print(n_entries)

1000000


In [23]:
#restrict to just 1 million events
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Range(1000000)

In [24]:
df_gst_GEM21_11b_all.Count().GetValue()

1000000

In [25]:
#apply definitions for new lepton variables
df_gst_GEM21_11b_all = define_df_gst_lep_vars(df_gst_GEM21_11b_all)

In [26]:
df_gst_GEM21_11b_all = define_df_gst_hadron_vars(df_gst_GEM21_11b_all)

In [27]:
df_gst_GEM21_11b_all = define_df_gst_hadrons_by_pdg(df_gst_GEM21_11b_all)

In [28]:
df_gst_GEM21_11b_all = define_df_gst_pi0decay(df_gst_GEM21_11b_all)

In [29]:
#Make a dataframe with default acceptances applied. Note the change in name for the dataframe!
df_gst_GEM21_11b_DefA = define_df_gst_hadron_acceptance(df_gst_GEM21_11b_all)
df_gst_GEM21_11b_DefA = define_df_gst_photon_acceptance(df_gst_GEM21_11b_DefA)
df_gst_GEM21_11b_DefA = define_df_gst_hadron_sums(df_gst_GEM21_11b_DefA,sfx=["i","f","fa"])

In [30]:
df_gst_GEM21_11b_DefA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_DefA,suffix_list=["i"],cname="i")
df_gst_GEM21_11b_DefA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_DefA,suffix_list=["f"],cname="f")
df_gst_GEM21_11b_DefA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_DefA,cname="fa_all",
                                                          suffix_list=["fa_proton","fa_neutron",
                                                                       "fa_piplus","fa_piminus",
                                                                       "fa_pi0_ph"])
df_gst_GEM21_11b_DefA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_DefA,
                                                         cname="fa_chhad",
                                                         suffix_list=["fa_proton","fa_piplus","fa_piminus"])

In [31]:
#Make a dataframe with acceptance without angular cuts applied.
df_gst_GEM21_11b_KEA = define_df_gst_hadron_acceptance(df_gst_GEM21_11b_all,thetaz_max=[10,10,10,10])
df_gst_GEM21_11b_KEA = define_df_gst_photon_acceptance(df_gst_GEM21_11b_KEA,thetaz_max=10.)
df_gst_GEM21_11b_KEA = define_df_gst_hadron_sums(df_gst_GEM21_11b_KEA,sfx=["i","f","fa"])

In [32]:
df_gst_GEM21_11b_KEA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_KEA,suffix_list=["i"],cname="i")
df_gst_GEM21_11b_KEA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_KEA,suffix_list=["f"],cname="f")
df_gst_GEM21_11b_KEA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_KEA,cname="fa_all",
                                                          suffix_list=["fa_proton","fa_neutron",
                                                                       "fa_piplus","fa_piminus",
                                                                       "fa_pi0_ph"])
df_gst_GEM21_11b_KEA = define_df_gst_momentum_imbalance(df_gst_GEM21_11b_KEA,
                                                         cname="fa_chhad",
                                                         suffix_list=["fa_proton","fa_piplus","fa_piminus"])

In [33]:
df_gst_GEM21_11b_all.Filter("ptl>0.4").Count().GetValue()

112392

In [25]:
#can print all of our columns here
for n in df_gst_GEM21_11b_all.GetColumnNames():
    print(n)

A
DXSec
Ef
Ef_K0
Ef_Kminus
Ef_Kplus
Ef_neutron
Ef_pi0
Ef_pi0_ph1
Ef_pi0_ph2
Ef_piminus
Ef_piplus
Ef_proton
Ei
Ei_K0
Ei_Kminus
Ei_Kplus
Ei_neutron
Ei_pi0
Ei_piminus
Ei_piplus
Ei_proton
El
En
Ev
EvRF
KPS
Q2
Q2s
W
Ws
XSec
Z
amnugamma
calresp0
cc
charm
coh
cthf
cthl
dfr
dis
em
energy_transfer
fspl
hitnuc
hitqrk
iev
imd
imdanh
kef
kef_K0
kef_Kminus
kef_Kplus
kef_neutron
kef_pi0
kef_piminus
kef_piplus
kef_proton
kei
kei_K0
kei_Kminus
kei_Kplus
kei_neutron
kei_pi0
kei_piminus
kei_piplus
kei_proton
massf
massf_K0
massf_Kminus
massf_Kplus
massf_neutron
massf_pi0
massf_piminus
massf_piplus
massf_proton
massi
massi_K0
massi_Kminus
massi_Kplus
massi_neutron
massi_pi0
massi_piminus
massi_piplus
massi_proton
mec
nc
neu
neut_code
nf
nfem
nfk0
nfkm
nfkp
nfn
nfother
nfp
nfpi0
nfpim
nfpip
ni
niem
nik0
nikm
nikp
nin
niother
nip
nipi0
nipim
nipip
nuance_code
nuel
p4vec_pi0_ph1
p4vec_pi0_ph2
pdgf
pdgi
pdl
pf
pf_K0
pf_Kminus
pf_Kplus
pf_neutron
pf_pi0
pf_piminus
pf_piplus
pf_proton
pi
pi_K0
pi_Kminus
pi_Kpl

In [26]:
#make a quick check to see that the proton indices are filled, and sorted by KE
rows=10
disp = df_gst_GEM21_11b_all.Range(rows).Display(["pdgf","massf","kef","kef_proton"],rows,100)
disp.Print()

+-----+------+------------+---------------+---------------+
| Row | pdgf | massf      | kef           | kef_proton    | 
+-----+------+------------+---------------+---------------+
| 0   | 2212 | 0.93827208 | 0.43649404    | 0.43649404    | 
|     | 211  | 0.13957018 | 1.8244414     | 0.053096929   | 
|     | 2112 | 0.93956541 | 0.095797126   |               | 
|     | 2112 | 0.93956541 | 0.048370546   |               | 
|     | 2112 | 0.93956541 | 0.015021265   |               | 
|     | 2112 | 0.93956541 | 0.018512205   |               | 
|     | 2212 | 0.93827208 | 0.053096929   |               | 
|     | 2112 | 0.93956541 | 0.16068890    |               | 
|     | 2112 | 0.93956541 | 0.049633567   |               | 
+-----+------+------------+---------------+---------------+
| 1   | 2212 | 0.93827208 | 0.30096254    | 0.30096254    | 
|     | 111  | 0.13497660 | 0.81738266    |               | 
|     | 111  | 0.13497660 | 1.7637369     |               | 
+-----+------+------------+

In [34]:
#define bins for a 2D plot in log-log space
etransfer_bins = array.array('d',np.logspace(np.log10(0.05),np.log10(4),100).tolist())
q2transfer_bins = array.array('d',np.logspace(np.log10(0.02),np.log10(10),100).tolist())

In [38]:
#create Q2 versus omega plot
h_lep_q2vw = df_gst_GEM21_11b_all.Histo2D(("","",len(etransfer_bins)-1,etransfer_bins,len(q2transfer_bins)-1,q2transfer_bins),
                                          "energy_transfer","ptl")

In [36]:
h_lep_q2vw.SetTitle("All events;Energy Transfer (GeV);Q^{2} (GeV/c)^{2}")
h_lep_q2vw.Draw("colz")
c.SetLogx()
c.SetLogy()
c.Draw()

In [30]:
#now, make a filter based on acceptance
df_gst_GEM21_11b_accept = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && abs(thetazl)<{lep_accept_angle}")

In [31]:
h_lep_q2vw_accept = df_gst_GEM21_11b_accept.Histo2D(("","",len(etransfer_bins)-1,etransfer_bins,
                                            len(q2transfer_bins)-1,q2transfer_bins),
                                      "energy_transfer","Q2")

In [32]:
h_lep_q2vw_accept.SetTitle("Events passing electron acceptance;Energy Transfer (GeV);Q^{2} (GeV/c)^{2}")
h_lep_q2vw_accept.Draw("colz")
c.SetLogx()
c.SetLogy()
c.Draw()

In [33]:
#reset canvas to linear scale
c.SetLogy(0)
c.SetLogx(0)

In [34]:
#Make 1D lepton pt histogram for QE, MEC..., but for those passing our lepton acceptance
h_lep_pt_QEL = df_gst_GEM21_11b_accept.Filter("qel==1").Histo1D(("","",100,0,3.0),"ptl")
h_lep_pt_MEC = df_gst_GEM21_11b_accept.Filter("mec==1").Histo1D(("","",100,0,3.0),"ptl")

In [35]:
h_lep_pt_QEL.SetLineColor(ROOT.kBlue)
h_lep_pt_MEC.SetLineColor(ROOT.kRed)
h_lep_pt_QEL.Draw()
h_lep_pt_MEC.Draw("same")
c.Draw()

In [36]:
#Make plot of KE for all protons
#we can select the protons based on the pdg code

h_kef_p_all = df_gst_GEM21_11b_all.Histo1D(("","",100,0,1.0),"kef_proton")

In [37]:
h_kef_p_all.Draw()
c.Draw()

In [38]:
#Make plot of KE for leading proton

#Now instead of plotting for all protons, we can plot for just the 'leading' (most energetic) proton
h_kef_p_max = df_gst_GEM21_11b_all.Define("kef_proton_max","Max(kef[pdgf==2212])").Histo1D(("","",100,0,1.0),"kef_proton_max")

In [39]:
h_kef_p_max.Draw()
c.Draw()

In [40]:
#Make plot of KE for leading proton, but let's remove events where the proton is very low energy (below 60 MeV)

#Now instead of plotting for all protons, we can plot for just the 'leading' (most energetic) proton
h_kef_p_max = df_gst_GEM21_11b_all.Define("kef_proton_max","Max(kef[pdgf==2212])").Filter("kef_proton_max>0.060").Histo1D(("","",100,0,1.0),"kef_proton_max")

In [41]:
h_kef_p_max.Draw()
c.Draw()